In [8]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import os
import pymongo
import datetime as dt   
import html, csv
import sys
import re
import csv
import zipfile
import pandas as pd
import shutil
import math
import time

In [2]:
# Store all the required urls into a dictionary list by type
url_List = []
url_dict = {}
url_dict["type"] = "news"
url_dict["url"] = "https://mars.nasa.gov/news/"
url_List.append(url_dict)
url_dict = {}
url_dict["type"] = "space"
url_dict["url"] = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_List.append(url_dict)
url_dict = {}
url_dict["type"] = "weather"
url_dict["url"] = "https://twitter.com/marswxreport?lang=en"
url_List.append(url_dict)
url_dict = {}
url_dict["type"] = "facts"
url_dict["url"] = "http://space-facts.com/mars/"
url_List.append(url_dict)
url_dict = {}
url_dict["type"] = "hemispheres"
url_dict["url"] = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
url_List.append(url_dict)

print(url_List)

[{'type': 'news', 'url': 'https://mars.nasa.gov/news/'}, {'type': 'space', 'url': 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'}, {'type': 'weather', 'url': 'https://twitter.com/marswxreport?lang=en'}, {'type': 'facts', 'url': 'http://space-facts.com/mars/'}, {'type': 'hemispheres', 'url': 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'}]


In [3]:
# Store the hemisphere word links into a list
hemisphere_List = ["Cerberus Hemisphere Enhanced", "Schiaparelli Hemisphere Enhanced", "Syrtis Major Hemisphere Enhanced", "Valles Marineris Hemisphere Enhanced"]

print(hemisphere_List)


['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']


In [4]:
for item in url_List:
    print(item["type"])
    print(item["url"])

news
https://mars.nasa.gov/news/
space
https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars
weather
https://twitter.com/marswxreport?lang=en
facts
http://space-facts.com/mars/
hemispheres
https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars


In [9]:
def init_browser():
    executable_path = {'executable_path': 'C:/Users/Janie228/SCHOOL/Browser_Drivers/chromedriver'}
    return Browser('chrome', **executable_path, headless=False)

In [6]:
# Initialize mars list to store data to upsert into mongodb
marsdict = {}
browser = init_browser()

# Loop thru the scrape list to scrape each site and parse according to the if criteria
for item in url_List:
#     print(item["type"])
#     print(item["url"])
    # If item is one of the three types blow, start off without scraping, else scrape
    if (item["type"] == "hemispheres") or (item["type"] == "space") or (item["type"] == "facts"):
        # If criteria met, visit site, click to result link, parse, store in dictionary
        if (item["type"] == "hemispheres"):
            sphereList = []

            for link in hemisphere_List:
                soup = ''
                i = 0
                sphereDict = {}
                sphereDict["img_title"] = link.replace(" Enhanced", "")
                
                while (soup == '') and (i < 3):
                    try:
                        browser.visit(item["url"])
                        time.sleep(5)
                        browser.click_link_by_partial_text(link)
                        time.sleep(5)
                        html = browser.html
                        soup = bs(html, "html.parser")

                        sphereDict["img_url"] = (soup.find("div", class_ = "downloads")).a["href"]
                        sphereList.append(sphereDict)  
                    except:
                        print("Connection refused by the server..")
                        print("Let me sleep for 5 seconds")
                        print("ZZzzzz...")
                        print(marsdict)
                        time.sleep(25)
                        print("Was a nice sleep, now let me continue...")
                        i = i + 1
                        continue
            
            marsdict["marsHemisphere"] = sphereList
                
        elif item["type"] == "facts":
            fact_table = pd.read_html(item["url"])[0]
            marsdict["marsFacts"] = dict(zip(fact_table[0] , fact_table[1]))

        elif item["type"] == "space":
            soup = ''
            i = 0
            
            while (soup == '') and (i < 3):
                try:
                    browser.visit(item["url"])
                    time.sleep(1)
                    browser.click_link_by_partial_text("FULL IMAGE")
                    time.sleep(3)
                    browser.click_link_by_partial_text("more info")

                    html = browser.html
                    soup = bs(html, "html.parser")

                    marsdict["img_releaseDate"] = (soup.find("h2", class_ = "category_title")).span.text.replace("|", "").strip()
                    marsdict["img_featuredTitle"] = soup.find("h1", class_ = "article_title").text.replace("\t", "").replace("\n", "").strip()
                    marsdict["img_featuredUrl"] = f"{'https://www.jpl.nasa.gov'}{(soup.find('figure', class_='lede')).a['href']}"
                except:
                    print("Connection refused by the server..")
                    print("Let me sleep for 5 seconds")
                    print("ZZzzzz...")
                    print(marsdict)
                    time.sleep(25)
                    print("Was a nice sleep, now let me continue...")
                    i = i + 1
                    continue 
    
    else:
        soup = ''
        i = 0

        while (soup == '') and (i < 3):
            try:
                # Visit site and scrape html
                browser.visit(item["url"])

                html = browser.html
                soup = bs(html, "html.parser")

                # If criteria met, parse and store in dictionary
                if item["type"] == "news":              
                    marsdict["newsTitle"] = soup.find("div", class_ = "content_title").text.strip()
                    marsdict["newsDesc"] = soup.find("div", class_ = "rollover_description_inner").text.strip()

                elif item["type"] == "weather":
                    browser.click_link_by_partial_text("Home")
                    marsdict["marsWeather"]  = (soup.find("p", class_ = "TweetTextSize")).text.split(weather.a.text)[0]
            #         marsdict["marsWeather"] = weather
            except:
                print("Connection refused by the server..")
                print("Let me sleep for 5 seconds")
                print("ZZzzzz...")
                print(marsdict)
                time.sleep(25)
                print("Was a nice sleep, now let me continue...")
                i = i + 1
                continue 

browser.quit()
print(marsdict)


Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
{'newsTitle': 'After a Reset, Curiosity Is Operating Normally', 'newsDesc': 'Curiosity has returned to science operations and is once again exploring the clay unit.', 'img_releaseDate': 'December 9, 2010', 'img_featuredTitle': 'Mars Odyssey All Stars: Bacolor Crater', 'img_featuredUrl': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA13664_hires.jpg'}
Was a nice sleep, now let me continue...
{'newsTitle': 'After a Reset, Curiosity Is Operating Normally', 'newsDesc': 'Curiosity has returned to science operations and is once again exploring the clay unit.', 'img_releaseDate': 'December 9, 2010', 'img_featuredTitle': 'Mars Odyssey All Stars: Bacolor Crater', 'img_featuredUrl': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA13664_hires.jpg', 'marsFacts': {'Equatorial Diameter:': '6,792 km', 'Polar Diameter:': '6,752 km', 'Mass:': '6.42 x 10^23 kg (10.7% Earth)', 'Moons:': '2 (Phobos & Deimos)', 'Or

In [7]:
#   <table>
#     <thead>
#       <tr>
#         <th>Description</th>
#         <th>Value</th>
#       </tr>
#     </thead>
#     <tbody>
#           {% for row in mars.mars_dict %}
#         <tr>
#             <td>{{ row }}</td>
#             <td>{{ mars.mars_dict[row] }}</td>
#         </tr>
#           {% endfor %}

#     </tbody>
#   </table>
         